In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from utils import PkdbModel, measurement_type_data, filter_out, convert_unit, unstring
plt.style.use('seaborn-whitegrid')
import numpy as np
from collections import namedtuple

from IPython.display import display, HTML

In [19]:
def multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell    '''
    return HTML(
         '<table><tr style="background-color:white;">' +
         ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
         '</tr></table>'
     )

def inferences(substance):
        return[
    Inference(
        name=f"{substance}_clearance",
        filter_out= {"unit":["milliliter / meter ** 2 / minute"]},
        measurement_type = "clearance",
    ),
    Inference(
        name=f"{substance}_timecourse",
        
        filter_out= {"unit":["mole / liter"],"unit_intervention":["gram / meter ** 2"]},
        measurement_type = "concentration"
    ),
    Inference(
        name=f"{substance}_thalf",
        filter_out= {"unit_intervention":["gram / meter ** 2"]},
        measurement_type = "thalf"
    ), 
    Inference(
        name=f"{substance}_tmax",
        filter_out= {"unit_intervention":["gram / meter ** 2"]},
        measurement_type = "tmax"
    ),
    Inference(
        name=f"{substance}_vd",
        filter_out= {"unit_intervention":["gram / meter ** 2"]},
        measurement_type = "vd"
    ),
     Inference(
        name=f"{substance}_auc_inf",
        filter_out= {"unit_intervention":["gram / meter ** 2"]},
        measurement_type = "auc_inf"
    ),
     Inference(
        name=f"{substance}_auc_end",
        filter_out= {"unit_intervention":["gram / meter ** 2"]},
        measurement_type = "auc_end"
    ),
    Inference(
        name=f"{substance}_kel",
        filter_out= {"unit_intervention":["gram / meter ** 2"]},
        measurement_type = "kel"
    ),
]

def paracetamol_idx(data):
    return (data.substance_intervention == 'paracetamol') \
           & (data.substance == 'paracetamol') \
           & (data['tissue'] == 'plasma')

def paracetamol_related_idx(data,substance):
    return (data.substance_intervention == 'paracetamol') \
           & (data.substance == substance) \
           & (data['tissue'] == 'plasma')

In [20]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.to_numeric()
complete_paracetamol = all_complete.data[paracetamol_idx(all_complete.data)]
complete_paracetamol["inferred"] = False

/home/mkoenig/envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [21]:
Inference = namedtuple("Inference",["name","filter_out","measurement_type"])

In [22]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.to_numeric()
substances = ["paracetamol","paracetamol glucuronide","paracetamol sulfate","paracetamol cysteine","paracetamol mercapturate"]

for substance in substances:
    complete_substance = all_complete.data.loc[paracetamol_related_idx(all_complete.data,substance)]
    complete_substance["inferred"] = False
    
    for inference in inferences(substance):
        print(inference.name)
        data = measurement_type_data(complete_substance,inference.measurement_type)
        data_class = PkdbModel(inference.name,destination="3-inferred")
        data_class.data = data
        [data_class.filter_out(k,v)for k,v in inference.filter_out.items()]
        data_class.infer_from_interventions()
        data_class.infer_from_outputs()
        data_class.save()

        t1 = pd.DataFrame(data_class.data.groupby(["unit", "unit_intervention"]).apply(len), columns=["count"])
        t2 = pd.DataFrame(data_class.data.groupby(["inferred"]).apply(len), columns=["count"])
        t3 = pd.DataFrame(data_class.data.groupby(["calculated"]).apply(len), columns=["count"])
        display(multi_table([t1,t2,t3]))

/home/mkoenig/envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


paracetamol_clearance


paracetamol_timecourse


paracetamol_thalf


paracetamol_tmax


...

paracetamol mercapturate_tmax


paracetamol mercapturate_vd


paracetamol mercapturate_auc_inf


paracetamol mercapturate_auc_end


paracetamol mercapturate_kel
